In [ ]:
import pandas as pd
from keras import models
from keras import layers
from keras.preprocessing import text
from keras import utils

from sklearn.feature_extraction.text import FeatureHasher

In [ ]:
dfs = pd.read_csv('train', chunksize=20000)

for df in dfs:
    break

## First approach

In [ ]:
df.nunique()

In [ ]:
df[["site_id", "site_domain", "site_category",
    "app_id", "app_domain", "app_category",
    "device_id", "device_ip", "device_model",
    "device_type", "device_conn_type"]].head()

In [ ]:
t_site_id = text.Tokenizer(num_words=None)
t_site_domain = text.Tokenizer(num_words=None)
t_site_category = text.Tokenizer(num_words=None)
t_app_id = text.Tokenizer(num_words=None)
t_app_domain = text.Tokenizer(num_words=None)
t_app_category = text.Tokenizer(num_words=None)
t_device_id = text.Tokenizer(num_words=None)
t_device_model = text.Tokenizer(num_words=None)

t_site_id.fit_on_texts(df['site_id'])
t_site_domain.fit_on_texts(df['site_domain'])
t_site_category.fit_on_texts(df['site_category'])
t_app_id.fit_on_texts(df['app_id'])
t_app_domain.fit_on_texts(df['app_domain'])
t_app_category.fit_on_texts(df['app_category'])
# t_device_id.fit_on_texts(df['device_id'])
t_device_model.fit_on_texts(df['device_model'])


x_site_id = t_site_id.texts_to_matrix(df['site_id'])
x_site_domain = t_site_domain.texts_to_matrix(df['site_domain'])
x_site_category = t_site_category.texts_to_matrix(df['site_category'])
x_app_id = t_app_id.texts_to_matrix(df['app_id'])
x_app_domain = t_app_domain.texts_to_matrix(df['app_domain'])
x_app_category = t_app_category.texts_to_matrix(df['app_category'])
# x_device_id = t_device_id.texts_to_matrix(df['device_id'])
x_device_model = t_device_model.texts_to_matrix(df['device_model'])

x = [df['banner_pos'], x_site_id, x_site_domain,
     x_site_category, x_app_id, x_app_domain,
     x_app_category, x_device_model]

In [ ]:
i = [layers.Input(shape=(1,), name='banner_pos'),
     layers.Input(shape=(x_site_id.shape[1],), name='site_id'),
     layers.Input(shape=(x_site_domain.shape[1],), name='site_domain'),
     layers.Input(shape=(x_site_category.shape[1],), name='site_category'),
     layers.Input(shape=(x_app_id.shape[1],), name='app_id'),
     layers.Input(shape=(x_app_domain.shape[1],), name='app_domain'),
     layers.Input(shape=(x_app_category.shape[1],), name='app_category'),
     # layers.Input(shape=(x_device_id.shape[1],), name='device_id'),
     layers.Input(shape=(x_device_model.shape[1],), name='device_model')]

h = layers.concatenate(i)
o = layers.Dense(1, activation='sigmoid')(h)

model = models.Model(inputs=i, outputs=[o])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x=x,
          y=df['click'],
          validation_split=0.1,
          epochs=3)

## Second Approach

In [ ]:
hasher = FeatureHasher(input_type='string', n_features=80000)

features = ["site_id", "site_domain", "site_category",
            "app_id", "app_domain", "app_category",
            "device_id", "device_ip", "device_model"]

i1 = layers.Input(shape=(X.shape[1],), name='categorical_features')
i2 = layers.Input(shape=(1,), name='banner_pos')

h = layers.concatenate([i1, i2])
h = layers.Dense(1024, activation='sigmoid')(h)
o = layers.Dense(1, activation='sigmoid')(h)

model = models.Model(inputs=[i1, i2], outputs=[o])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
for df in pd.read_csv('train', chunksize=50000):
    X = hasher.transform(df[features].apply(lambda r: '|'.join(r.values), axis=1).values)

    model.fit(x=[X, df['banner_pos']],
              y=df['click'],
              validation_split=0.1,
              epochs=3)
    
    break